<a href="https://colab.research.google.com/github/bkkaggle/lm-experiments/blob/master/lm_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

In [0]:
DIST_BUCKET="gs://tpu-pytorch/wheels"
TORCH_WHEEL="torch-1.15-cp36-cp36m-linux_x86_64.whl"
TORCH_XLA_WHEEL="torch_xla-1.15-cp36-cp36m-linux_x86_64.whl"
TORCHVISION_WHEEL="torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl"

# Install Colab TPU compat PyTorch/TPU wheels and dependencies
!pip uninstall -y torch torchvision
!gsutil cp "$DIST_BUCKET/$TORCH_WHEEL" .
!gsutil cp "$DIST_BUCKET/$TORCH_XLA_WHEEL" .
!gsutil cp "$DIST_BUCKET/$TORCHVISION_WHEEL" .
!pip install "$TORCH_WHEEL"
!pip install "$TORCH_XLA_WHEEL"
!pip install "$TORCHVISION_WHEEL"
!sudo apt-get install libomp5

In [0]:
!pip install git+https://github.com/bkkaggle/transformers fire wandb line_profiler

import os

!git clone https://github.com/NVIDIA/apex
os.chdir('apex')
!pip install -v --no-cache-dir ./
os.chdir('../')

!git clone https://github.com/bkkaggle/lm-experiments.git
os.chdir('lm-experiments/')

!wandb login 08283aec9b92f676724219169ea3c28cf41c85cc

In [0]:
!git pull

In [0]:
!python finetune.py --train_dataset_path ./data/all-the-news-subset/npr/npr-distilgpt2-train.pkl --val_dataset_path ./data/all-the-news-subset/npr/npr-distilgpt2-test.pkl --model_type gpt2 --checkpoint distilgpt2 --optimizer SGD --lr 1e-3 --batch_size 64 --epochs 1

In [0]:
# !python run_pplm.py -B military --cond_text "The potato" --length 100 --gamma 1.5 --num_iterations 3 --num_samples 1 --stepsize 0.03 --window_length 5 --kl_scale 0.01 --gm_scale 0.99 --sample
!python run_pplm.py -D sentiment --class_label 2 --cond_text "The potato" --length 100 --gamma 1.5 --num_iterations 3 --num_samples 1 --stepsize 0.03 --window_length 5 --kl_scale 0.01 --gm_scale 0.99 --sample

In [0]:
!python run_pplm.py -B ./data/pplm/sample-bow.txt --cond_text "The potato" --length 100 --gamma 1.5 --num_iterations 3 --num_samples 1 --stepsize 0.03 --window_length 5 --kl_scale 0.01 --gm_scale 0.99 --sample

In [0]:
import pandas as pd
df = pd.DataFrame([['positive', 'text'], ['negative', 'text2']], columns=['label', 'text'])
df.to_csv('data.tsv', sep='\t', index=False)

In [0]:
!python run_pplm_discrim_train.py --dataset SST --pretrained_model distilgpt2 --epochs 1 --log_interval 1000 --cached --save_model
!python run_pplm_discrim_train.py --dataset generic --dataset_fp ./data.tsv --pretrained_model distilgpt2 --epochs 1 --log_interval 1000 --cached --save_model
!python run_pplm.py -D generic --discrim_weights ./SST_classifier_head_epoch_1.pt --discrim_meta ./SST_classifier_head_meta.json --class_label 2 --cond_text "The potato" --length 100 --gamma 1.5 --num_iterations 3 --num_samples 1 --stepsize 0.03 --window_length 5 --kl_scale 0.01 --gm_scale 0.99 --sample